#  deep network

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [3]:
# Load dataset (handwritten digits)
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step 


In [4]:
# Normalize input data
x_train = x_train.reshape(-1, 28*28) / 255.0  # Flatten to 1D
x_test = x_test.reshape(-1, 28*28) / 255.0

In [5]:
# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


In [6]:
# Initialize Weights & Biases
np.random.seed(42)
weights_1 = np.random.randn(28*28, 128) * 0.01  # Input → Hidden Layer 1
bias_1 = np.zeros((1, 128))

weights_2 = np.random.randn(128, 64) * 0.01  # Hidden Layer 1 → Hidden Layer 2
bias_2 = np.zeros((1, 64))

weights_3 = np.random.randn(64, 10) * 0.01  # Hidden Layer 2 → Output
bias_3 = np.zeros((1, 10))

In [7]:
# Activation Functions
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)


In [8]:
# Training Parameters
learning_rate = 0.01
epochs = 10
batch_size = 32


In [9]:
# Training Loop
for epoch in range(epochs):
    for i in range(0, x_train.shape[0], batch_size):
        X_batch = x_train[i:i+batch_size]
        Y_batch = y_train[i:i+batch_size]

        # Forward Propagation
        hidden_1_input = np.dot(X_batch, weights_1) + bias_1
        hidden_1_output = relu(hidden_1_input)

        hidden_2_input = np.dot(hidden_1_output, weights_2) + bias_2
        hidden_2_output = relu(hidden_2_input)

        final_input = np.dot(hidden_2_output, weights_3) + bias_3
        final_output = softmax(final_input)

        # Compute Loss (Cross-Entropy)
        loss = -np.mean(Y_batch * np.log(final_output + 1e-8))

        # Backpropagation
        error_output = final_output - Y_batch
        d_weights_3 = np.dot(hidden_2_output.T, error_output)
        d_bias_3 = np.sum(error_output, axis=0, keepdims=True)

        error_hidden_2 = np.dot(error_output, weights_3.T) * relu_derivative(hidden_2_output)
        d_weights_2 = np.dot(hidden_1_output.T, error_hidden_2)
        d_bias_2 = np.sum(error_hidden_2, axis=0, keepdims=True)

        error_hidden_1 = np.dot(error_hidden_2, weights_2.T) * relu_derivative(hidden_1_output)
        d_weights_1 = np.dot(X_batch.T, error_hidden_1)
        d_bias_1 = np.sum(error_hidden_1, axis=0, keepdims=True)

        # Update Weights & Biases
        weights_1 -= learning_rate * d_weights_1
        bias_1 -= learning_rate * d_bias_1

        weights_2 -= learning_rate * d_weights_2
        bias_2 -= learning_rate * d_bias_2

        weights_3 -= learning_rate * d_weights_3
        bias_3 -= learning_rate * d_bias_3

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss}")


Epoch 1/10, Loss: 0.002707791241651848
Epoch 2/10, Loss: 0.0007457718243550569
Epoch 3/10, Loss: 0.00022550068368956512
Epoch 4/10, Loss: 0.000539492553072174
Epoch 5/10, Loss: 0.00033017614171527954
Epoch 6/10, Loss: 0.000698447287987444
Epoch 7/10, Loss: 0.00012354822612908978
Epoch 8/10, Loss: 4.8900798292600083e-05
Epoch 9/10, Loss: 3.979601965333988e-05
Epoch 10/10, Loss: 5.876249570324764e-05


In [10]:
# Test Accuracy
hidden_1_test = relu(np.dot(x_test, weights_1) + bias_1)
hidden_2_test = relu(np.dot(hidden_1_test, weights_2) + bias_2)
final_test = softmax(np.dot(hidden_2_test, weights_3) + bias_3)

accuracy = np.mean(np.argmax(final_test, axis=1) == np.argmax(y_test, axis=1))
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 97.01%
